In [4]:
#importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import seaborn as sns


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import make_scorer, recall_score, log_loss
from sklearn.metrics import average_precision_score

In [5]:
#loading datas

data_train ="C:/Users/shuda/project/train.csv"
data_test = "C:/Users/shuda/project/test.csv"
loan_train = pd.read_csv(data_train)
loan_test = pd.read_csv(data_test)

In [6]:
loan_train_copy = loan_train.copy()
#data preview

loan_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
loan_train.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [8]:
loan_train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [9]:
#checking for missing values
loan_train.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

# Data Cleaning & Preparation

For train data set

In [10]:
#using forward fill, to place 1 or 0 in the place of missing data

loan_train['Credit_History'].fillna(method='ffill', inplace=True)

In [11]:
#filling columns using the median of the values

median_loan = loan_train['Loan_Amount_Term'].median()
loan_train['Loan_Amount_Term'].fillna((median_loan), inplace=True)

median_loan_amount = loan_train['LoanAmount'].median()
loan_train['LoanAmount'].fillna((median_loan_amount), inplace=True)



In [12]:
# Count the values to know which occurs most frequently

loan_train['Self_Employed'].value_counts()

loan_train['Married'].mode()

loan_train['Gender'].mode()

0    Male
Name: Gender, dtype: object

In [13]:
#filling the mode

loan_train['Self_Employed'].fillna('No', inplace=True)

loan_train['Dependents'].fillna(0, inplace=True)

loan_train['Married'].fillna('Yes', inplace=True)

loan_train['Gender'].fillna('Male', inplace=True)


In [14]:
#check if there is still some data missing

loan_train.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

For test data set

In [15]:
# A preview of missing data in the testing set

loan_test.isna().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [16]:
# forward filling

loan_test['Credit_History'].fillna(method='ffill', inplace=True)

# filling missing data using median

median_loan_test = loan_test['Loan_Amount_Term'].median()
loan_test['Loan_Amount_Term'].fillna((median_loan_test), inplace=True)


median_loan_amount_test = loan_test['LoanAmount'].median()
loan_test['LoanAmount'].fillna((median_loan_amount_test), inplace=True)

# filling data using mode

loan_test['Self_Employed'].fillna('No', inplace=True)

loan_test['Dependents'].fillna(0, inplace=True)


loan_test['Gender'].fillna('Male', inplace=True)



In [219]:
#checking for missing data

loan_test.isna().values.any()

False

In [17]:
#checking for duplicate values

loan_train.duplicated().values.any()

loan_test.duplicated().values.any()

False

# Data Pre-processing

In [18]:
loan_train.drop('Loan_ID', axis=1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [19]:
#first identify all categorical columns & pass into a variable

objectlist_train = loan_train.select_dtypes(include = "object").columns


One-hot encoding

In [20]:
objectlist_train

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

oh_encoder = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Loan_Status','Gender','Education','Self_Employed','Property_Area','Married'])], 
                               remainder='passthrough')
df_onehot = oh_encoder.fit_transform(loan_train)

print(df_onehot[:6])

[[0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 'LP001002' '0' 5849
  0.0 128.0 360.0 1.0]
 [1.0 0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 'LP001003' '1' 4583
  1508.0 128.0 360.0 1.0]
 [0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 'LP001005' '0' 3000
  0.0 66.0 360.0 1.0]
 [0.0 1.0 0.0 1.0 0.0 1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 'LP001006' '0' 2583
  2358.0 120.0 360.0 1.0]
 [0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 'LP001008' '0' 6000
  0.0 141.0 360.0 1.0]
 [0.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 'LP001011' '2' 5417
  4196.0 267.0 360.0 1.0]]


# AMMENDMENDS 

In [22]:
categorical_columns = list(loan_train.select_dtypes(object).columns)

In [23]:
def encode_categorical_data(dataframe: pd.DataFrame, one_hot_encoder: OneHotEncoder) -> pd.DataFrame:
    categorical_columns = get_categorical_column_names(dataframe)
    encoded_categorical_df = one_hot_encoder.transform(dataframe[categorical_columns])
    return encoded_categorical_df

def get_one_encoder(dataframe: pd.DataFrame) -> OneHotEncoder:
    categorical_columns = get_categorical_column_names(dataframe)
    one_hot_encoder = OneHotEncoder()
    one_hot_encoder.fit(dataframe[categorical_columns])
    return one_hot_encoder

In [24]:
#converting categorical values to numbers

to_numeric = {'Male': 1, 'Female': 2,
'Yes': 1, 'No': 2,
'Graduate': 1, 'Not Graduate': 2,
'Urban': 3, 'Semiurban': 2,'Rural': 1,
'Y': 1, 'N': 0,
'3+': 3}

# adding the new numeric values from the to_numeric variable to both datasets
loan_train = loan_train.applymap(lambda lable: to_numeric.get(lable) if lable in to_numeric else lable)
loan_train = loan_train.applymap(lambda lable: to_numeric.get(lable) if lable in to_numeric else lable)

Modelling

In [25]:
X = loan_train[['Gender','Credit_History','Education','Married','Self_Employed','Property_Area']]
y = loan_train["Loan_Status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
def train_model(loan_train, max_depth=2):
    # Split data
    X = loan_train[['Gender','Credit_History','Education','Married','Self_Employed','Property_Area']]
    y = loan_train[["Loan_Status"]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit model
    model = RandomForestRegressor(max_depth=max_depth)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f"Test mse = {mse}, Test RMSE = {rmse}, Random forest max depth = {max_depth}")
    return model, mse, rmse

In [240]:
_ = train_model(loan_train, max_depth=2)

Test mse = 0.15093628680058344, Test RMSE = 0.3885051953328082, Random forest max depth = 2


C:\Users\shuda\AppData\Local\Temp\ipykernel_49244\3676535765.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


In [27]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

(491, 6)
(123, 6)
(123,)
(491,)


In [28]:
LR = LogisticRegression()
LR.fit(X_train, y_train)

y_predict = LR.predict(X_test)

#  prediction Summary by species
print(classification_report(y_test, y_predict))

# Accuracy score
LR_SC = accuracy_score(y_predict,y_test)
print('accuracy is',accuracy_score(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.95      0.49      0.65        43
           1       0.78      0.99      0.87        80

    accuracy                           0.81       123
   macro avg       0.87      0.74      0.76       123
weighted avg       0.84      0.81      0.79       123

accuracy is 0.8130081300813008


In [258]:
#train the models using default parameters
models = []
models.append(('Logistic Regression', LogisticRegression(solver='liblinear', random_state = 0,
                                                         class_weight='balanced')))
#models.append(('SVC', SVC(kernel = 'linear', random_state = 0)))
#models.append(('Kernel SVM', SVC(kernel = 'rbf', random_state = 0)))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)))
models.append(('Gaussian NB', GaussianNB()))
models.append(('Decision Tree Classifier',
               DecisionTreeClassifier(criterion = 'entropy', random_state = 0)))
models.append(('Random Forest', RandomForestClassifier(
    n_estimators=100, criterion = 'entropy', random_state = 0)))
#Evaluating Model Results:
acc_results = []
auc_results = []
names = []
# set table to table to populate with performance results
col = ['Algorithm', 'ROC AUC Mean', 'ROC AUC STD', 
       'Accuracy Mean', 'Accuracy STD']
model_results = pd.DataFrame(columns=col)
i = 0
# Evaluate each model using k-fold cross-validation:
for name, model in models:
    kfold = model_selection.KFold(
        n_splits=2)
# accuracy scoring:
    cv_acc_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    # roc_auc scoring:
    cv_auc_results = model_selection.cross_val_score(  
    model, X_train, y_train, cv=kfold, scoring='roc_auc')
    acc_results.append(cv_acc_results)
    auc_results.append(cv_auc_results)
    names.append(name)
    model_results.loc[i] = [name,
                             round(cv_auc_results.mean()*100, 2),
                             round(cv_auc_results.std()*100, 2),
                             round(cv_acc_results.mean()*100, 2),
                             round(cv_acc_results.std()*100, 2)]
    i += 1
    
model_results.sort_values(by=['ROC AUC Mean'], ascending=False)  

,Algorithm,ROC AUC Mean,ROC AUC STD,Accuracy Mean,Accuracy STD
0,Logistic Regression,75.48,0.28,73.33,3.82
2,Gaussian NB,73.17,0.25,80.44,1.67
4,Random Forest,70.51,1.87,78.20,1.88
1,KNN,66.40,2.79,76.17,1.38
3,Decision Tree Classifier,63.60,3.71,75.15,1.27


In [38]:

import pickle


In [39]:
with open('model_pickle','wb') as f:
    pickle.dump(LR,f)

In [40]:
with open('model_pickle','rb')as f:
    mp=pickle.load(f)